In [963]:
import os
import re
from skimage.filters import threshold_otsu, gaussian
from skimage import measure, color, morphology
from skimage.transform import resize
from skimage.io import imread, imsave
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [964]:
def extract_and_preprocess_leads(image_file, parent_folder, output_folder):
    """
    Extracts individual leads from an ECG image, preprocesses them to remove noise 
    and keep only the ECG signal, and saves the smaller resulting images
    Args:
      image_file: Name of the input image file.
      parent_folder: Path to the folder containing the image file.
      output_folder: Path to the folder where the processed lead images will be saved.
    """
    # Read the image
    image = imread(os.path.join(parent_folder, image_file))

    # Dividing the ECG leads (adjust coordinates based on your image format)
    # Dividing the ECG leads (adjust coordinates to start a few pixels later)
    start_offset = 30  # Adjust this value as needed

    Lead_1 = image[300:600, 150 + start_offset:643]
    Lead_2 = image[300:600, 646 + start_offset:1135]
    Lead_3 = image[300:600, 1140 + start_offset:1626]
    Lead_4 = image[300:600, 1630 + start_offset:2125]
    Lead_5 = image[600:900, 150 + start_offset:643]
    Lead_6 = image[600:900, 646 + start_offset:1135]
    Lead_7 = image[600:900, 1140 + start_offset:1626]
    Lead_8 = image[600:900, 1630 + start_offset:2125]
    Lead_9 = image[900:1200, 150 + start_offset:643]
    Lead_10 = image[900:1200, 646 + start_offset:1135]
    Lead_11 = image[900:1200, 1140 + start_offset:1626]
    Lead_12 = image[900:1200, 1630 + start_offset:2125]

    Leads=[Lead_1,Lead_2,Lead_3,Lead_4,Lead_5,Lead_6,Lead_7,Lead_8,Lead_9,Lead_10,Lead_11,Lead_12]

     # Extract the original filename without extension
    base_filename = os.path.splitext(image_file)[0]

    # Extract the class label from the base filename
    class_label = re.match(r'([^\(]+)', base_filename).group(1) 
    folder_name = re.sub('.jpg', '', image_file)
    output_path = os.path.join(output_folder, folder_name)

    for x, lead_img in enumerate(Leads):
        # Convert to grayscale
        grayscale = color.rgb2gray(lead_img)
        # Smooth the image
        blurred_image = gaussian(grayscale, sigma=0.7)
        # Thresholding
        global_thresh = threshold_otsu(blurred_image)
        binary_global = blurred_image < global_thresh  
        # Resize
        binary_global = resize(binary_global, (150,220))  # Smaller size
       
        # Find contours to isolate the ECG signal
        contours = measure.find_contours(binary_global, 0.8)
        contours_shape = sorted([x.shape for x in contours])[::-1][0:1]
        # Create a blank image to draw the extracted signal
        extracted_signal = np.zeros_like(binary_global)
        for contour in contours:
            if contour.shape in contours_shape:
                # Draw the contour on the blank image
                for point in contour:
                    x_coord, y_coord = int(point[1]), int(point[0])
                    extracted_signal[y_coord, x_coord] = 1  # Set pixel to white
        
       
        # Create the output filename with the desired convention
        output_filename = f"{base_filename}_lead_{x+1}.png"  
        output_path = os.path.join(output_folder, output_filename)  # Save directly to output_folder
        # Save the extracted signal image
        imsave(output_path, extracted_signal)


In [965]:
# Example usage:
input_folder = '.'  # Input folder
output_folder = './Output'  # Desired output folder

for class_label in ['AB', 'HMI', 'MI', 'Normal']:
    class_path = os.path.join(input_folder, class_label)
    for filename in os.listdir(class_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            extract_and_preprocess_leads(filename, class_path, output_folder)

/var/folders/92/yrzy6lgs0hx75g3j4tsnw8n40000gp/T/ipykernel_54915/561443712.py:68: UserWarning: ./Trial/Output/HB(20)_lead_1.png is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  imsave(output_path, extracted_signal)
/var/folders/92/yrzy6lgs0hx75g3j4tsnw8n40000gp/T/ipykernel_54915/561443712.py:68: UserWarning: ./Trial/Output/HB(20)_lead_2.png is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  imsave(output_path, extracted_signal)
/var/folders/92/yrzy6lgs0hx75g3j4tsnw8n40000gp/T/ipykernel_54915/561443712.py:68: UserWarning: ./Trial/Output/HB(20)_lead_3.png is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  imsave(output_path, extracted_signal)
/var/folders/92/yrzy6lgs0hx75g3j4tsnw8n40000gp/T/ipykernel_54915/561443712.py:68: UserWarning: ./Trial/Output/HB(20)_l